In [ ]:
"""
Let's consider the following model. We seek to predict how many retweets and likes a news headline will receive 
on Twitter. The main input to the model will be the headline itself, as a sequence of words, but to spice things 
up, our model will also have an auxiliary input, receiving extra data such as the time of day when the headline 
was posted, etc. The model will also be supervised via two loss functions. Using the main loss function earlier 
in a model is a good regularization mechanism for deep models.

The main input will receive the headline, as a sequence of integers (each integer encodes a word). The integers 
will be between 1 and 10,000 (a vocabulary of 10,000 words) and the sequences will be 100 words long.
"""

In [1]:
import keras
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(x)

Using TensorFlow backend.


In [2]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [3]:
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [4]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

In [6]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

In [9]:
import numpy as np

MODEL_INPUT_BATCH_SIZE=32

headline_data = np.random.uniform(low=1, high=9000, size=(MODEL_INPUT_BATCH_SIZE, 100)).astype(np.float32)
headline_data.shape
lstm_data = headline_data.reshape(MODEL_INPUT_BATCH_SIZE,1,100)
additional_data = np.random.uniform(low=1, high=9000, size=(MODEL_INPUT_BATCH_SIZE, 5)).astype(np.float32)
labels = np.random.randint(0, 1 + 1, size=(MODEL_INPUT_BATCH_SIZE, 1))

In [10]:
# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=MODEL_INPUT_BATCH_SIZE)

Epoch 1/50
32/32 [==============================] - 1s 46ms/step - loss: 8.0876 - main_output_loss: 7.9484 - aux_output_loss: 0.6964
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 4.1568 - main_output_loss: 4.0295 - aux_output_loss: 0.6362
Epoch 3/50
32/32 [==============================] - 0s 6ms/step - loss: 4.1438 - main_output_loss: 4.0295 - aux_output_loss: 0.5714
Epoch 4/50
32/32 [==============================] - 0s 5ms/step - loss: 4.1265 - main_output_loss: 4.0295 - aux_output_loss: 0.4849
Epoch 5/50
32/32 [==============================] - 0s 5ms/step - loss: 4.1052 - main_output_loss: 4.0295 - aux_output_loss: 0.3783
Epoch 6/50
32/32 [==============================] - 0s 6ms/step - loss: 4.0837 - main_output_loss: 4.0295 - aux_output_loss: 0.2708
Epoch 7/50
32/32 [==============================] - 0s 6ms/step - loss: 4.0681 - main_output_loss: 4.0295 - aux_output_loss: 0.1931
Epoch 8/50
32/32 [==============================] - 0s 6ms/step - loss: 4.0